In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from os import listdir

import scipy.cluster.hierarchy as sch
from scipy import stats

from keras.models import *
from keras.layers import *
from tensorflow.python.keras import backend as k

from matplotlib.colors import ListedColormap
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.model_selection import cross_val_score

%matplotlib inline

Using TensorFlow backend.


In [2]:
def folder_finder(path):

    file_list = []

    for j in glob(path+"*.csv"):
        file_list.append(j)

    return file_list

def read_file(path):

    data = pd.read_csv(path, na_values = 'NaN', keep_default_na = False) 
    return data

def pd_to_np(data):
    
    if type(data) == np.ndarray:
      print('Data is already in numpy format!')
    else:
      data = data.values
      #print('Pandas to Numpy done!')

    return data


def string_to_index(activity_label):

    activity_class =[]
    har_class = {
                    'Cook':0,
                    'Eat':1,
                    'Phone':2,
                    'Read':3,
                    'Watch_TV':4
                }
    for label in activity_label:
        activity_class.append(har_class[label[0]])

    return activity_class


def data_loader(path, split=0.3):
    x = y = []
    feature_list = []
    
    pd_data = read_file(path)

    for i in pd_data:
        feature_list.append(i)

    selectData = pd_data.loc[:, feature_list[:-1]]
    activityLabel = pd_data.loc[:, ['activity']]
    x = pd_to_np(selectData)
    x = StandardScaler().fit_transform(x)
    y = string_to_index(activityLabel.values)
    y = np.asarray(y) 
    y = y.astype('int32')
    return x,y

def full_dataset(file_list):

    x=y = np.asarray([])
    x_temp =  y_temp =  []

    for i in range(len(file_list)):
        x_temp, y_temp = data_loader(file_list[i])

        if i == 0:
            x = x_temp
            y = y_temp
        else:
            x = np.concatenate([x, x_temp],axis=0)
            y  = np.concatenate([y ,y_temp],axis=0)
    return x,y

In [3]:
file_list = folder_finder("../Dataset/")
print(file_list)
print('Total files:',len(file_list))
X,y= full_dataset(file_list)
print(X.shape)
print(y.shape)

['../Dataset\\new_dataset.csv']
Total files: 1
(447530, 36)
(447530,)


In [4]:
### EXaMPLE OF DATA FOR A SINGLE INDIVIDUAL ###
data = np.array(X,dtype=np.float64)
#data['Target'] = np.array(y, dtype=np.float64)
### STANDARDIZE DATA ###

def std(serie): return (serie - serie.mean(axis=0))/serie.std(axis=0)
def reverse_std(serie): return serie.std(axis=0)+serie.mean(axis=0)

reverse_data = np.array([data.std(axis=0)+data.mean(axis=0),data])
data = np.array([(data-data.mean(axis=0))/data.std(axis=0),data])
data.shape

C:\Users\Mahbuba\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


(2, 447530, 36)

# DIMENSIONALITY REDUCTION

In [13]:
import keras
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior() 

input_layer = Input(shape=data)
encoder = TimeDistributed(Dense(200, activation='relu'))(input_layer)
encoder = TimeDistributed(Dense(50, activation="relu"))(encoder)

decoder = TimeDistributed(Dense(50, activation='relu'))(encoder)
decoder = TimeDistributed(Dense(200, activation='relu'))(decoder)

out = TimeDistributed(Dense(3))(decoder)

autoencoder = Model(inputs=input_layer, outputs=out)
autoencoder.compile(optimizer='adam', loss='mean_squared_error')

AttributeError: module 'tensorflow' has no attribute 'placeholder'

In [ ]:
autoencoder.summary()

In [ ]:
autoencoder.fit(data[:10],data[:10], epochs=100, shuffle=True, verbose=2)

In [ ]:
### TRY TO RECONSTRUCT ERRORS ###

predictions = autoencoder.predict(data[11][np.newaxis,:,:]) * reverse_data[11]

mse = np.mean(np.power(data[11] - predictions[0], 2), axis=1)

plt.figure(figsize=(16,6))
plt.scatter(y=mse,x=range(data.shape[1]))
plt.title('reconstruction error '+person_id[11])

In [ ]:
def reconstructe_error(series, reverse_data):
    
    predictions = autoencoder.predict(series[np.newaxis,:,:]) * reverse_data
    mse = np.mean(np.power(series - predictions[0], 2), axis=1)
    return mse

# CORRELATION CLUSTERING

In [ ]:
### COMPUTE CORRELATION MATRIX ###

df = pd.DataFrame([reconstructe_error(series,reverse) for series,reverse in zip(data[10:],reverse_data[10:])]).T
print(df.shape)

plt.figure(figsize=(6,6))
plt.imshow(df.corr())
plt.xticks(range(len(person_id[10:])), person_id[10:])
plt.yticks(range(len(person_id[10:])), person_id[10:])

In [ ]:
### HIERACHICAL CLUSTERING ###

X = df.corr().values
d = sch.distance.pdist(X)
L = sch.linkage(d, method='ward')
ind = sch.fcluster(L, d.max()*0.99, 'distance')
columns = [df.columns.tolist()[i] for i in list((np.argsort(ind)))]
df = df.reindex(columns, axis=1)

plt.figure(figsize=(6,6))
plt.imshow(df.corr())
plt.xticks(range(len(person_id[10:])), person_id[10:])
plt.yticks(range(len(person_id[10:])), person_id[10:])

In [ ]:
### PLOT DENDROGRAM ###

plt.figure(figsize=(8,6))
dendrogram = sch.dendrogram(L, labels=person_id[10:])
plt.hlines(d.max()*0.99,0,np.max(dendrogram['icoord']))

# KOLMOGOROV SMIRNOV CLUSTERING

In [ ]:
### COMPUTE KOLMOGOROV SMIRNOV MATRIX ###

df = pd.DataFrame([reconstructe_error(series,reverse) for series,reverse in zip(data[10:],reverse_data[10:])]).T
print(df.shape)

results = []
for i in range(len(df.columns)):
    kf_test = df.apply(lambda x: stats.ks_2samp(df.iloc[:,i].values, x)[0],axis=0)
    results.append(kf_test.tolist())

plt.figure(figsize=(6,6))
plt.imshow(results)
plt.xticks(range(len(person_id[10:])), person_id[10:])
plt.yticks(range(len(person_id[10:])), person_id[10:])

In [ ]:
### HIERACHICAL CLUSTERING ###

X = np.asarray(results)
d = sch.distance.pdist(X)
L = sch.linkage(d, method='ward')
ind = sch.fcluster(L, d.max()*0.99, 'distance')

plt.figure(figsize=(6,6))
plt.imshow(X[:,np.argsort(ind)])
plt.xticks(range(len(person_id[10:])), person_id[10:])
plt.yticks(range(len(person_id[10:])), person_id[10:])

In [ ]:
### PLOT DENDROGRAM ###

plt.figure(figsize=(8,6))
dendrogram = sch.dendrogram(L, labels=person_id[10:])
plt.hlines(d.max()*0.95,0,np.max(dendrogram['icoord']))